# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 4

- Выбрать метрику оценки качества и обосновать выбор
- Разработать baseline (может быть несколько алгоритмов)
- Реализовать выбранное решение/я
- Протестировать работу baseline
- Выбрать итоговое решение для дальнейшей оптимизации и обосновать выбор

In [ ]:
import warnings


# ----------------
# Data processing
# ----------------
import dill

import numpy as np
import numpy.typing as npt

import pandas as pd

from tqdm.auto import tqdm


# ---------------------
# RecSys models imports
# ---------------------
from lightfm import LightFM

from rectools import Columns
from rectools.dataset import Dataset as RTDataset
from rectools.models import (
    PopularModel,
    LightFMWrapperModel,
    implicit_knn,
)
from implicit import nearest_neighbours
from mab2rec import BanditRecommender, LearningPolicy


# --------------
# Plotting libs
# --------------
# import plotly.express as px
# import matplotlib.pyplot as plt
# import seaborn as sns
# import shap


# -------------------
# Metrics Evaluation
# -------------------
from metrics import RecommenderMetrics


warnings.filterwarnings("ignore")
# sns.set_theme(style="whitegrid")

RANDOM_STATE = 42

# Load Data

In [ ]:
data_path = "../data_closed/"

In [ ]:
# # Загружаем таблицу айтемов
# with open(data_path + "df_items.dill", "rb") as f:
#     df_items = dill.load(f)

# Загружаем таблицу данных для моделей первого уровня
with open(data_path + "base_models_data.dill", "rb") as f:
    base_models_data = dill.load(f)

# Загружаем таблицу данных для ранкера
with open(data_path + "ranker_data.dill", "rb") as f:
    ranker_data = dill.load(f)

# Загружаем таблицу тестовых данных
with open(data_path + "test_df.dill", "rb") as f:
    test_df = dill.load(f)

Вспомним структуру данных

In [ ]:
print("df_items")
display(df_items.dtypes)


print("\nbase_models_data")
display(base_models_data.dtypes)


print("\nranker_data")
display(ranker_data.dtypes)


print("\ntest_df")
display(test_df.dtypes)

Выборки по пользователям (USER_ID), участвующим в разных этапах обучения моделей

In [ ]:
# Уникальные айдишники пользователей в таблицах
base_users = base_models_data["user_id"].unique()
# save
with open(data_path + "base_users.dill", "wb") as f:
    dill.dump(base_users, f)

ranker_users = ranker_data["user_id"].unique()
# save
with open(data_path + "ranker_users.dill", "wb") as f:
    dill.dump(ranker_users, f)

test_users = test_df["user_id"].unique()
# save
with open(data_path + "test_users.dill", "wb") as f:
    dill.dump(test_users, f)

# Пользователи, которым надо выдавать пресказания для обучения ранкера,
# т.е. присутствуют и в base_models_data и в ranker_data (base to ranker users)
b2r_users = np.array(list((set(base_users) & set(ranker_users))))
display("b2r_users", b2r_users, b2r_users.shape)
# save
with open(data_path + "b2r_users.dill", "wb") as f:
    dill.dump(b2r_users, f)


# на оставшихся пользователях ранкер обучаться не будет
# на них просто не будет скоров
ranker_only_users = np.array(list(set(ranker_users) - set(base_users)))
display("ranker_only_users", ranker_only_users, ranker_only_users.shape)
# save
with open(data_path + "ranker_only_users.dill", "wb") as f:
    dill.dump(ranker_only_users, f)


# Пользователи из test_df, которым будут выданы
# таргетирвонные рекомондации
bNr2t_users = np.array(list((set(base_users) | set(ranker_users)) & set(test_users)))
display("bNr2t_users", bNr2t_users, bNr2t_users.shape)
# save
with open(data_path + "bNr2t_users.dill", "wb") as f:
    dill.dump(bNr2t_users, f)

# Пользователи, которые присутствуют только в test_df (cold_users)
test_only_users = np.array(list(set(test_users) - (set(base_users) | set(ranker_users))))
display("test_only_users", test_only_users, test_only_users.shape)
# save
with open(data_path + "test_only_users.dill", "wb") as f:
    dill.dump(test_only_users, f)

# Обучение моделей первого уровня для обучения ранкера

In [ ]:
models_path = "../models/"

Модели максимально простые, основанные на взаимодействиях пользователей и айтемов.

Фитчи айтемов оставим для переранжирования.

## Качество работы моделей будем оценивать следующим образом

In [ ]:
# Проверим качество на тестовой выборке
# Берем только пользователей, которые присутствуют
# в base и test выборках
b2t_users = np.array(list(set(test_users) & (set(base_users))))
b2t_users, b2t_users.shape

In [ ]:
# Пользователей много, так что выберем
# 100 тысяч пользователей, на которых расчитаем метрики
b2t_users = np.random.choice(
    b2t_users,
    size=10**5,
    replace=False,
)
b2t_users, b2t_users.shape

Выделим часть таблицы, на которой будем сверяться

In [ ]:
metrics_df_tmp= test_df[test_df["user_id"].isin(b2t_users)]

## Rectools Dataset

Используется библиотека `rectools`, так что dataset должен содержать 4 основные колонки: 
- user_id,
- item_id,
- datetime,
- weight,

где weight &mdash; максимально хорошо описывает вес предмета (важность взаимодейстия) в момент взаимодействия с ним пользователем. 

Из таблицы взаимодействий такими параметрами являются:
- `weight`: описывает общий вес взаимодействия (кол-во взаимодействий с предметом / общее число взаимодействий), 
- `cum_weight`: вес предмета в зависимости от номера вхождения этого предмета (отношение номера входжения предмета к общему числу взаимодействий с предметом, умноженное на рейтинг этого предмета), 
- `rel_weight`: вес соответствующий товару при каждом новом взаимодействии пользователя с товаром (отношения числа взаимодейсвий с предметом к общему число взаимодействий со всеми предметами на момент записи взаимодействия)

В контекстке учета времени для получения веса интеракции, на мой взгляд, самым валидным параметром является `rel_weight`.

In [ ]:
# Изменим датасет `base_models_data`
# Оставим только нужные колонки и переименуем под стандарт `rectools`
base_models_data = base_models_data[
    [
        "user_id",
        "item_id",
        "dt",
        "rel_weight",
    ]
]

base_models_data = base_models_data.rename(
    columns={
        "user_id": Columns.User,
        "item_id": Columns.Item,
        "dt": Columns.Datetime,
        "rel_weight": Columns.Weight,
    }
)

# Создадим датасет взаимодействий
current_dataset = RTDataset.construct(
    interactions_df=base_models_data,
)

## Rectools PopularModel

In [ ]:
popular_model = PopularModel()

In [ ]:
popular_model.fit(current_dataset)

%clear

In [ ]:
# Save model
with open(models_path + "popular_model.dill", "wb") as f:
    dill.dump(popular_model, f)

In [ ]:
# Load model
with open(models_path + "popular_model.dill", "rb") as f:
    popular_model = dill.load(f)

### Test PopularModel

In [ ]:
candidates_pop = popular_model.recommend(
    b2t_users,
    current_dataset,
    # выдаем 10 кандидатов
    k=10,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_pop = candidates_pop.rename(
    columns={
        "score": "pop_score",
        "rank": "pop_rank",
    }
)
candidates_pop.head(3)

In [ ]:
predictions = (
    candidates_pop[candidates_pop["pop_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "pop_recs"})
)

metrics_df_tmp = pd.merge(metrics_df_tmp, predictions, how="left", on="user_id")

#### Calculate metrics

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="pop_recs",
)

## Rectools Implicit


In [ ]:
knn_impl_cos_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=50)
)

knn_impl_bm25_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.BM25Recommender(K=50)
)

knn_impl_tfidf_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.TFIDFRecommender(K=50)
)

In [ ]:
# Fit models
knn_impl_cos_k50.fit(current_dataset)
knn_impl_bm25_k50.fit(current_dataset)
knn_impl_tfidf_k50.fit(current_dataset)

%clear

In [ ]:
# Save models

with open(models_path + "knn_impl_cos_k50.dill", "wb") as f:
    dill.dump(knn_impl_cos_k50, f)

with open(models_path + "knn_impl_bm25_k50.dill", "wb") as f:
    dill.dump(knn_impl_bm25_k50, f)

with open(models_path + "knn_impl_tfidf_k50.dill", "wb") as f:
    dill.dump(knn_impl_tfidf_k50, f)

In [ ]:
# Load models

with open(models_path + "knn_impl_cos_k50.dill", "rb") as f:
    knn_impl_cos_k50 = dill.load(f)

with open(models_path + "knn_impl_bm25_k50.dill", "rb") as f:
    knn_impl_bm25_k50 = dill.load(f)

with open(models_path + "knn_impl_tfidf_k50.dill", "rb") as f:
    knn_impl_tfidf_k50 = dill.load(f)

### Test Cosine

In [ ]:
candidates_cos = knn_impl_cos_k50.recommend(
    b2t_users,
    current_dataset,
    # выдаем 10 кандидатов
    k=10,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_cos = candidates_cos.rename(
    columns={
        "score": "cos_score",
        "rank": "cos_rank",
    }
)
candidates_cos.head(3)

In [ ]:
predictions = (
    candidates_cos[candidates_cos["cos_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "cos_recs"})
)
metrics_df_tmp = pd.merge(metrics_df_tmp, predictions, how="left", on="user_id")

#### Calculate metrics

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="cos_recs",
)

### Test BM25

In [ ]:
candidates_bm25 = knn_impl_bm25_k50.recommend(
    b2t_users,
    current_dataset,
    # выдаем 10 кандидатов
    k=10,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_bm25 = candidates_bm25.rename(
    columns={
        "score": "bm25_score",
        "rank": "bm25_rank",
    }
)
candidates_bm25.head(3)

In [ ]:
predictions = (
    candidates_bm25[candidates_bm25["bm25_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "bm25_recs"})
)

metrics_df_tmp = pd.merge(
    metrics_df_tmp,
    predictions,
    how="left",
    on="user_id",
)

#### Calculate metrics

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="bm25_recs",
)

### Test TFIDF

In [ ]:
candidates_tfidf = knn_impl_tfidf_k50.recommend(
    b2t_users,
    current_dataset,
    # выдаем 10 кандидатов
    k=10,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_tfidf = candidates_tfidf.rename(
    columns={
        "score": "tfidf_score",
        "rank": "tfidf_rank",
    }
)
candidates_tfidf.head(3)

In [ ]:
predictions = (
    candidates_tfidf[candidates_tfidf["tfidf_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "tfidf_recs"})
)

metrics_df_tmp = pd.merge(
    metrics_df_tmp,
    predictions,
    how="left",
    on="user_id",
)

#### Calculate metrics

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="tfidf_recs",
)

## Rectools LightFM

In [ ]:
# Задаем модель
lfm_model = LightFMWrapperModel(
    LightFM(
        no_components=64,
        learning_rate=0.1,
        loss="warp",
        max_sampled=7,
    ),
    epochs=20,
    num_threads=6,
    verbose=1,
)

In [ ]:
lfm_model.fit(dataset=current_dataset)
%clear

In [ ]:
# Save model
with open(models_path + "lfm_model.dill", "wb") as f:
    dill.dump(lfm_model, f)

In [ ]:
# Load model
with open(models_path + "lfm_model.dill", "rb") as f:
    lfm_model = dill.load(f)

### Test LightFM

In [ ]:
candidates_lfm = lfm_model.recommend(
    b2t_users,
    current_dataset,
    # выдаем 10 кандидатов
    k=10,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_lfm = candidates_lfm.rename(
    columns={
        "score": "lfm_score",
        "rank": "lfm_rank",
    }
)
candidates_lfm.head(3)

In [ ]:
predictions = (
    candidates_lfm[candidates_lfm["lfm_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "lfm_recs"})
)

metrics_df_tmp = pd.merge(metrics_df_tmp, predictions, how="left", on="user_id")

#### Calculate metrics

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="lfm_recs",
)

## Bandit Recommender

In [ ]:
mab_model = BanditRecommender(
    LearningPolicy.ThompsonSampling(),
    top_k=10,
    n_jobs=-1,
)

In [ ]:
# Загружаем таблицу данных для моделей первого уровня
with open(data_path + "base_models_data.dill", "rb") as f:
    base_models_data = dill.load(f)

In [ ]:
mab_data = base_models_data[base_models_data["user_id"].isin(b2r_users)]
mab_data = mab_data[(mab_data["u_total_inter"] > 20)]
mab_data["binary_weight"] = (mab_data["ui_inter"] > 2).astype(int)
mab_data = mab_data[
    [
        "user_id",
        "item_id",
        "dt",
        "binary_weight",
    ]
]
mab_data

In [ ]:
time_windows = []
min_date = mab_data["dt"].min()
max_date = mab_data["dt"].max()


cur_min = min_date
left, right = cur_min, cur_min + pd.Timedelta(hours=1.5)
chunk = mab_data[mab_data["dt"].between(left, right, inclusive="left")]

mab_model.fit(
    decisions=chunk["item_id"],
    rewards=chunk["binary_weight"],
)

print(f"Fitted: {left}, {right}")

cur_min = right

while right <= max_date:
    left, right = cur_min, cur_min + pd.Timedelta(hours=1.5)
    chunk = mab_data[mab_data["dt"].between(left, right, inclusive="left")]
    mab_model.partial_fit(
        decisions=chunk["item_id"],
        rewards=chunk["binary_weight"],
    )
    cur_min = right
    print(f"Fitted: {left}, {right}")


In [ ]:
# Save model
with open(models_path + "mab_model.dill", "wb") as f:
    dill.dump(mab_model, f)

In [ ]:
# Load model
with open(models_path + "mab_model.dill", "rb") as f:
    mab_model: BanditRecommender = dill.load(f)

In [ ]:
class KNNBanditRecommender:
    """
    Class for recommending items with Multi-Armed Bandit
    and knn model
    """

    def __init__(
        self,
        dataset: RTDataset,
        path_bandit_model: str,
        path_knn_model: str,
        path_popular_model: str,
    ):

        self.dataset = dataset

        with open(path_bandit_model, "rb") as f:
            self.mab_model: BanditRecommender = dill.load(f)

        with open(path_knn_model, "rb") as f:
            self.knn_model: implicit_knn = dill.load(f)

        with open(path_popular_model, "rb") as f:
            self.popular_model: implicit_knn = dill.load(f)

    def __get_arms_for_users(self, user_ids):
        candidates_knn = self.knn_model.recommend(
            user_ids,
            self.dataset,
            # выдаем 25 кандидатов
            # из которых будет выбирать бандит
            k=25,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )

        return candidates_knn[["user_id", "item_id"]]

    def predict(self, user_ids: npt.ArrayLike):

        recs = pd.DataFrame()
        cur_recs = pd.DataFrame()

        candidates_knn = self.__get_arms_for_users(user_ids)

        candidates_pop = self.popular_model.recommend(
            [user_ids[0]],
            self.dataset,
            # выдаем 50 кандидатов
            k=50,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )["item_id"].values

        print("KNN predicted")

        for user_id in tqdm(user_ids):
            try:
                filtered_arms = candidates_knn[candidates_knn["user_id"] == user_id][
                    "item_id"
                ].values
                if len(filtered_arms) < self.mab_model.top_k:
                    filtered_arms = np.concatenate(
                        [
                            filtered_arms,
                            np.random.choice(
                                candidates_pop,
                                size=25,
                                replace=False,
                            ),
                        ]
                    )

                self.mab_model.set_arms(filtered_arms)
                mab_recs = self.mab_model.recommend(return_scores=True)
                cur_recs["user_id"] = [user_id] * self.mab_model.top_k
                cur_recs["item_id"] = mab_recs[0]
                cur_recs["mab_score"] = mab_recs[1]
                cur_recs["mab_rank"] = [i for i in range(1, 11)]

                recs = pd.concat([recs, cur_recs])
            except Exception as e:
                print(filtered_arms)
                print(user_id)
                print(mab_recs)
                raise e

        return recs

In [ ]:
knn_bandit_model = KNNBanditRecommender(
    dataset=current_dataset,
    path_bandit_model=models_path + "mab_model.dill",
    path_knn_model=models_path + "knn_impl_bm25_k50.dill",
    path_popular_model=models_path + "popular_model.dill",
)

In [ ]:
predictions = knn_bandit_model.predict(b2t_users)
predictions

In [ ]:
candidates_mab = predictions
candidates_mab

In [ ]:
predictions = (
    candidates_mab[candidates_mab["mab_rank"] <= 10][["user_id", "item_id"]]
    .groupby(by="user_id")["item_id"]
    .apply(list)
    .reset_index()
    .rename(columns={"item_id": "mab_recs"})
)

metrics_df_tmp = pd.merge(metrics_df_tmp, predictions, how="left", on="user_id")

In [ ]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp,
    model_preds_col="mab_recs",
)

## Наблюдения по моделям первого уровня

|       **Модель**       | **Время обучения** | **Время иференса (100 тыс. пользователей)** | **ndcg** | **recall** |
|:----------------------:|:------------------:|:-------------------------------------------:|----------|------------|
|      **PopularK**      |        6.6 с       |                    0.8 с                    |  0.0126  |   0.0021   |
|  **knn_impl_cos_k50**  |       14.25 с      |                    12.5 с                   |  0.2566  |   0.0465   |
|  **knn_impl_bm25_k50** |       13.25 с      |                    11.5 с                   |  0.2517  |   0.0459   |
| **knn_impl_tdidf_k50** |       13.25 с      |                    11.5 с                   |  0.256   |   0.0461   |
| **LightFM (epoch=20)** |       7 м 15 с     |                     1 м                     |   0.165  |   0.0279   |
| **Bandit Recommender** |        14 м        |                     10 м                    |   0.147  |   0.0257   |

Из таблицы видно, что knn based алгоритмы хорошо и быстро обучаются на полном датасете, когда lightFM модель требует большего времени как для обучения, так и для инференса.

Bandit Recommender в коопе с KNN в принципе перспективная связка, но времени для обучения и инференса требует много, скорее всего откажусь от танного типа модели. 

DL модели в качестве моделей первого уровня решил не запускать: 
- мало временных зависимостей: все таки если логически размышлять, то блуждание пользователя по товарам в ленте тем более за 2 дня --- это слишком волатильный процесс
- так же для DL модели лучше использовать фитчи предметов или пользователей, но их было решено оставить под обучение ранкера

Для автоэнкодеров --- слишком большой датасет, было бы нормально, если пользователей можно было бы разнести по кластерам, но для этого у нас нет данных о самих пользователях.


### ИТОГО

Оставляем PopularModel, kNN-based методы и LightFM

## Получим рекомендации для обучения ранкера

In [ ]:
candidates_data_path = models_path + "candidates_data/"

### PopularModel


In [ ]:
# PopularModel
with open(models_path + "popular_model.dill", "rb") as f:
    popular_model: PopularModel = dill.load(f)

In [ ]:
candidates_pop = popular_model.recommend(
    b2r_users,
    current_dataset,
    # выдаем 20 кандидатов
    k=20,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)

candidates_pop = candidates_pop.rename(
    columns={
        "score": "pop_score",
        "rank": "pop_rank",
    }
)

candidates_pop

In [ ]:
# Save PopularModel candidates
with open(candidates_data_path + "candidates_pop.dill", "wb") as f:
    dill.dump(candidates_pop, f)

### Cosine Recommender

In [ ]:
# Cosine Recommender
with open(models_path + "knn_impl_cos_k50.dill", "rb") as f:
    knn_impl_cos_k50 = dill.load(f)

In [ ]:
candidates_cos = knn_impl_cos_k50.recommend(
    b2r_users,
    current_dataset,
    # выдаем 20 кандидатов
    k=20,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_cos = candidates_cos.rename(
    columns={
        "score": "cos_score",
        "rank": "cos_rank",
    }
)

candidates_cos

In [ ]:
# Save Cosine Model candidates
with open(candidates_data_path + "candidates_cos.dill", "wb") as f:
    dill.dump(candidates_cos, f)

### BM25 Recommender


In [ ]:
# BM25 Recommender
with open(models_path + "knn_impl_bm25_k50.dill", "rb") as f:
    knn_impl_bm25_k50 = dill.load(f)

In [ ]:
candidates_bm25 = knn_impl_bm25_k50.recommend(
    b2r_users,
    current_dataset,
    # выдаем 20 кандидатов
    k=20,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_bm25 = candidates_bm25.rename(
    columns={
        "score": "bm25_score",
        "rank": "bm25_rank",
    }
)
candidates_bm25

In [ ]:
# Save BM25 Model candidates
with open(candidates_data_path + "candidates_bm25.dill", "wb") as f:
    dill.dump(candidates_bm25, f)

### TFIDF Recommender


In [ ]:
# TFIDF Recommender
with open(models_path + "knn_impl_tfidf_k50.dill", "rb") as f:
    knn_impl_tfidf_k50 = dill.load(f)

In [ ]:
candidates_tfidf = knn_impl_tfidf_k50.recommend(
    b2r_users,
    current_dataset,
    # выдаем 20 кандидатов
    k=20,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_tfidf = candidates_tfidf.rename(
    columns={
        "score": "tfidf_score",
        "rank": "tfidf_rank",
    }
)
candidates_tfidf

In [ ]:
# Save TFIDF Model candidates
with open(candidates_data_path + "candidates_tfidf.dill", "wb") as f:
    dill.dump(candidates_tfidf, f)

### LightFM Recommender


In [ ]:
# LightFM Recommender
with open(models_path + "lfm_model.dill", "rb") as f:
    lfm_model = dill.load(f)

In [ ]:
candidates_lfm = lfm_model.recommend(
    b2r_users,
    current_dataset,
    # выдаем 20 кандидатов
    k=20,
    # рекомендуем уже просмотренные товары
    filter_viewed=False,
)
candidates_lfm = candidates_lfm.rename(
    columns={
        "score": "lfm_score",
        "rank": "lfm_rank",
    }
)
candidates_lfm

In [ ]:
# Save LightFM Model candidates
with open(candidates_data_path + "candidates_lfm.dill", "wb") as f:
    dill.dump(candidates_lfm, f)

# Сливаем всех кандидатов в одну таблицу

In [ ]:
models_path = "../models/"

In [ ]:
candidates_data_path = models_path + "candidates_data/"

Так как LightFM умеет работать с warm и cold пользователями (PopularModel была взята для тех же целей), а PopularModel имеет **плохой score** относительно остальных моделей и **сильно увеличивает размерность** получаемого датасета с кандидатами, то от кандидатов PopularModel решено отказаться

In [ ]:
with open(candidates_data_path + "candidates_cos.dill", "rb") as f:
    candidates_cos = dill.load(f)
    candidates_cos = candidates_cos[candidates_cos["cos_rank"] < 15]

with open(candidates_data_path + "candidates_bm25.dill", "rb") as f:
    candidates_bm25 = dill.load(f)
    candidates_bm25 = candidates_bm25[candidates_bm25["bm25_rank"] < 15]

with open(candidates_data_path + "candidates_tfidf.dill", "rb") as f:
    candidates_tfidf = dill.load(f)
    candidates_tfidf = candidates_tfidf[candidates_tfidf["tfidf_rank"] < 15]

with open(candidates_data_path + "candidates_lfm.dill", "rb") as f:
    candidates_lfm = dill.load(f)
    candidates_lfm = candidates_lfm[candidates_lfm["lfm_rank"] < 15]

# with open(candidates_data_path + "candidates_pop.dill", "rb") as f:
#     candidates_pop = dill.load(f)

In [ ]:
candidates_list = [
    candidates_cos,
    candidates_bm25,
    candidates_tfidf,
    candidates_lfm,
    # candidates_pop,
]

In [ ]:
for df in candidates_list:
    print(df.shape)

## Concatenate

In [ ]:
candidates = candidates_list[0].copy()

for df in candidates_list[1:]:
    candidates = pd.concat(
        [
            candidates.set_index(["user_id", "item_id"]),
            df.set_index(["user_id", "item_id"]),
        ],
        join="outer",
        axis=1,
    ).reset_index()

In [ ]:
#Check shape
candidates.shape

### Fill NaN

In [ ]:
default_values_merged = {
    "cos_score": candidates["cos_score"].min() - 0.01,
    "bm25_score": candidates["bm25_score"].min() - 0.01,
    "tfidf_score": candidates["tfidf_score"].min() - 0.01,
    "lfm_score": candidates["lfm_score"].min() - 0.01,
    "cos_rank": candidates["cos_rank"].max() + 1,
    "bm25_rank": candidates["bm25_rank"].max() + 1,
    "tfidf_rank": candidates["tfidf_rank"].max() + 1,
    "lfm_rank": candidates["lfm_rank"].max() + 1,
}

In [ ]:
candidates.fillna(default_values_merged, inplace=True)
candidates.head(10)

In [ ]:
# Checkpoint
with open(candidates_data_path + "candidates_full.dill", "wb") as f:
    dill.dump(candidates, f)